In [10]:
main_amenity = "undersea_hyperbaric_medicine"
target_amenities = " 'assisted_living_facility', 'doula', 'blood_and_plasma_donation_center' "
dist_in_meters=100

In [ ]:
sd.sql(
f""" WITH main AS ( 
SELECT 
    geometry AS geom, 
    addresses.locality as locality 
FROM places
WHERE 
    categories.primary = '{main_amenity}' 
    and ARRAY_CONTAINS(addresses.country, 'US') 
), 

amenities AS ( 
SELECT 
    geometry AS geom, 
    categories.primary as cat 
FROM places
WHERE 
    categories.primary IN ({target_amenities}) 
    and ARRAY_CONTAINS(addresses.country, 'US') 
) 

SELECT 
    COUNT(a.*) AS proximity_count, locality, array_distinct(array_agg(cat)) 
FROM main g 
JOIN amenities a ON ST_DWithin(g.geom, a.geom, {dist_in_meters}, True) 
group by locality 
order by 1 DESC 
""").show(5)

In [2]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [3]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=places/type=place/"
)

In [4]:
df.to_view("places")

In [11]:
sd.sql(f"""
SELECT 
    geometry AS geom, 
    addresses.locality as locality 
FROM places
WHERE 
    categories.primary = '{main_amenity}' 
    AND array_has(addresses.country, 'US')
""").show(3)

SedonaError: The expression to get an indexed field is only valid for `Struct`, `Map` or `Null` types, got List(Field { name: "element", data_type: Struct([Field { name: "freeform", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "locality", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "postcode", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "region", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "country", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }]), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} })